# Project 5: US College COVID-19 Analysis

##### Aziz Maredia, Katharine King, Manuel Sainz de la Peña, Max Bosse | DSIR-1019 | 12.23.20

## Problem Statement

## Data Sets & Dictionary

* [`nyt_college.csv`](./datasets/nyt_college.csv): Colleges and counts of Coronavirus case counts, [NYT Repository](https://github.com/nytimes/covid-19-data/tree/master/colleges)
* [`admissions1.csv`](./datasets/admissions1.csv): Data pulled from IPED's database, part 1, [IPED Database](https://nces.ed.gov/ipeds/)
* [`admissions2.csv`](./datasets/admissions2.csv): Data pulled from IPED's database, part 2, [IPED Database](https://nces.ed.gov/ipeds/)
* [`sports.csv`](./datasets/sports.csv): Data pulled from IPED's database, part 3, [IPED Database](https://nces.ed.gov/ipeds/)
* [`regions.csv`](./datasets/regions.csv): State region and sports division
* [`state_policies.csv`](./datasets/state_policies.csv): State Social Distancing and other Coronavirus policies as of August 11th, [Kaiser Family Foundation](https://github.com/KFFData/COVID-19-Data/tree/kff_master/State%20Policy%20Actions/State%20Social%20Distancing%20Actions)
* [`classplan.csv`]('./datasets/classplan.csv'): Dataset of colleges and their intended plan for the Fall of 2020 [The Chronicle of Higher Education](https://www.chronicle.com/article/heres-a-list-of-colleges-plans-for-reopening-in-the-fall/)
* [`combined_df.csv`]('./datasets/combined_df.csv): Merged Dataframe created in [intro_data_merging_cleaning]('./code/intro_data_merging_cleaning.ipnyb')

Target Column: Predict a University's classification into **greater_than_5**, which indicates whether a college has case counts that exceed 5% of the total enrolled population

## Dataframe Merging & Cleaning

#### Loading in NYT college covid data

In [1]:
import pandas as pd

data = pd.read_csv('./datasets/nyt_college.csv')
data.head()

,date,state,county,city,ipeds_id,college,cases,notes
0,2020-12-11,Texas,Taylor,Abilene,222178.0,Abilene Christian University,553.0,NaN
1,2020-12-11,Georgia,Tift,Tifton,138558.0,Abraham Baldwin Agricultural College,165.0,NaN
2,2020-12-11,California,San Francisco,San Francisco,108232.0,Academy of Art University,NaN,NaN
3,2020-12-11,Colorado,Alamosa,Alamosa,126182.0,Adams State University,108.0,NaN
4,2020-12-11,New York,Nassau,Garden City,188429.0,Adelphi University,60.0,NaN


#### Checking for null values and dropping row w/ with null 'ipeds_id'

In [2]:
data.isnull().sum()

date           0
state          0
county         3
city           0
ipeds_id       1
college        0
cases         70
notes       1916
dtype: int64

In [3]:
data.dropna(subset = ['ipeds_id'], inplace = True)
data.reset_index(inplace = True)

#### Dropping columns unnecessary for analysis

In [5]:
data.drop(columns = ['index', 'date', 'county', 'city', 'notes'], inplace = True)

#### Building function to create string of all 'ipeds_ids' separtated by commas 

#### This string will be used to easily search for an pull college admissions statistics from https://nces.ed.gov/ipeds/use-the-data

In [6]:
data['ids'] = [str(n) for n in data['ipeds_id'].astype(int)]

def get_commaed_list(column, max_length):
    
    number_of_lists = int(len(column)/max_length)+1
    start = 0
    
    for i in range(1,number_of_lists):
        print(f'list{i}')
        string = ''
        
        for n in range(start,(max_length*(i))):
            string = string + (str(column[n]) + ',')
        print(string)
        start = start + max_length
    
    for i in range(number_of_lists, number_of_lists+1):   
        print(f'list{i}')
        string = ''
        for n in range((max_length*(number_of_lists-1)),len(column)):
            string = string + (str(column[n]) + ',')
        print(string,)
    
    return

In [7]:
get_commaed_list(data['ids'], 400)

list1
222178,138558,108232,126182,188429,188438,168528,133872,138600,200697,100654,100724,188526,188580,138716,385415,128498,168546,210571,175342,237118,172918,188641,156189,210669,154642,168591,168607,210775,238193,447883,164447,240736,131159,164465,150066,217633,168740,222831,164492,172954,172963,109350,245865,197869,168786,126289,211088,104151,104160,107327,106458,440402,106467,404994,156222,156213,156231,201104,164562,100812,138901,132374,485768,100858,100830,173045,482149,143084,219000,143118,222983,222992,219602,446048,451149,451158,231420,457411,109785,164580,154688,109819,201195,150136,189015,189088,189097,132471,197911,160977,437103,168883,380359,164632,223232,164720,247065,175421,156286,234669,234696,197984,219709,238333,173124,137591,217721,154712,145619,230816,164739,156295,183789,189228,139144,154721,237181,173142,154749,173160,219718,132602,102429,137856,157553,110097,100937,200022,219046,106625,143288,223427,183822,211158,231554,237215,156392,201371,217749,142115,189413,

#### Loading in college admissions statistics dataframes

In [8]:
# could only pull 20 features at a time so had to pull twice

admissions = pd.read_csv('./datasets/admissions1.csv', index_col = 'UnitID')
print(admissions.shape)

admissions_2 = pd.read_csv('./datasets/admissions2.csv', index_col = 'UnitID')
print(admissions_2.shape)

(6527, 24)
(6527, 33)


#### Merging in first admissions statistics to base dataframe and dropping unnecessary columns

In [9]:
data = data.merge(admissions, left_on = data['ipeds_id'].astype(int), right_on = admissions.index, how = 'left')
data.head()

,key_0,state,ipeds_id,college,cases,ids,Institution Name,Institution (entity) name (HD2019),Institution name alias (HD2019),Fips County code (HD2019),...,Imputation/Status Flag For Applicants total (ADM2019),Admissions total (ADM2019),Imputation/Status Flag For Admissions total (ADM2019),Enrolled total (ADM2019),Imputation/Status Flag For Enrolled total (ADM2019),Admissions men (ADM2019),Imputation/Status Flag For Admissions men (ADM2019),Admissions women (ADM2019),Imputation/Status Flag For Admissions women (ADM2019),Unnamed: 24
0,222178,Texas,222178.0,Abilene Christian University,553.0,222178,Abilene Christian University,Abilene Christian University,,48441.0,...,R,6996.0,R,932.0,R,2558.0,R,4438.0,R,NaN
1,138558,Georgia,138558.0,Abraham Baldwin Agricultural College,165.0,138558,Abraham Baldwin Agricultural College,Abraham Baldwin Agricultural College,ABAC,13277.0,...,R,1475.0,R,919.0,R,653.0,R,822.0,R,NaN
2,108232,California,108232.0,Academy of Art University,NaN,108232,Academy of Art University,Academy of Art University,,6075.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,126182,Colorado,126182.0,Adams State University,108.0,126182,Adams State University,Adams State University,,8003.0,...,R,1749.0,R,399.0,R,711.0,R,1038.0,R,NaN
4,188429,New York,188429.0,Adelphi University,60.0,188429,Adelphi University,Adelphi University,,36059.0,...,R,10342.0,R,1132.0,R,3244.0,R,7098.0,R,NaN


#### Dropping columns unnecessary for analysis after merge

In [10]:
columns_to_drop = [
                    'key_0',
                    'Institution Name',
                    'Institution (entity) name (HD2019)',
                    'Institution name alias (HD2019)',
                    'Fips County code (HD2019)',
                    'State abbreviation (HD2019)',
                    'FIPS state code (HD2019)',
                    'Degree-granting status (HD2019)',
                    'Undergraduate level programs or courses are offered via distance education (IC2019)',
                    'Graduate level programs or courses are offered via distance education (IC2019)',
                    'Does not offer distance education opportunities (IC2019)',
                    'Distance education programs offered (IC2019)',
                    'Applicants total (ADM2019)',
                    'Imputation/Status Flag For Applicants total (ADM2019)',
                    'Imputation/Status Flag For Admissions total (ADM2019)',
                    'Imputation/Status Flag For Enrolled total (ADM2019)',
                    'Imputation/Status Flag For Admissions men (ADM2019)',
                    'Imputation/Status Flag For Admissions women (ADM2019)',
                    'Unnamed: 24'
                  ]

# 'Institution size category (HD2019)', 'Distance education courses offered (IC2019)'

In [11]:
data.drop(columns = columns_to_drop, inplace = True)

#### Merging in second admissions statistics to base dataframe and dropping unnecessary columns

In [12]:
data = data.merge(admissions_2, left_on = data['ipeds_id'], right_on = admissions_2.index, how = 'left')
data.head()

,key_0,state,ipeds_id,college,cases,ids,Institution size category (HD2019),Distance education courses offered (IC2019),Admissions total (ADM2019),Enrolled total (ADM2019),...,Imputation/Status Flag For ACT Composite 75th percentile score (ADM2019),ACT English 25th percentile score (ADM2019),Imputation/Status Flag For ACT English 25th percentile score (ADM2019),ACT English 75th percentile score (ADM2019),Imputation/Status Flag For ACT English 75th percentile score (ADM2019),ACT Math 25th percentile score (ADM2019),Imputation/Status Flag For ACT Math 25th percentile score (ADM2019),ACT Math 75th percentile score (ADM2019),Imputation/Status Flag For ACT Math 75th percentile score (ADM2019),Unnamed: 33
0,222178.0,Texas,222178.0,Abilene Christian University,553.0,222178,3.0,1.0,6996.0,932.0,...,R,19.0,R,28.0,R,18.0,R,26.0,R,NaN
1,138558.0,Georgia,138558.0,Abraham Baldwin Agricultural College,165.0,138558,2.0,1.0,1475.0,919.0,...,R,15.0,R,22.0,R,16.0,R,21.0,R,NaN
2,108232.0,California,108232.0,Academy of Art University,NaN,108232,3.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,126182.0,Colorado,126182.0,Adams State University,108.0,126182,2.0,1.0,1749.0,399.0,...,R,15.0,R,21.0,R,16.0,R,22.0,R,NaN
4,188429.0,New York,188429.0,Adelphi University,60.0,188429,3.0,1.0,10342.0,1132.0,...,R,22.0,R,29.0,R,19.0,R,26.0,R,NaN


#### Dropping columns unnecessary for analysis after merge

In [13]:
columns_to_drop2 = [
                    'key_0',
                    'Institution Name',
                    'Institution (entity) name (HD2019)',
                    'Institution name alias (HD2019)',
                    'Fips County code (HD2019)',
                    'Imputation/Status Flag For Number of first-time degree/certificate-seeking students submitting SAT scores (ADM2019)',
                    'Percent of first-time degree/certificate-seeking students submitting SAT scores (ADM2019)',
                    'Imputation/Status Flag For Percent of first-time degree/certificate-seeking students submitting SAT scores (ADM2019)',
                    'Imputation/Status Flag For Number of first-time degree/certificate-seeking students submitting ACT scores (ADM2019)',
                    'Percent of first-time degree/certificate-seeking students submitting ACT scores (ADM2019)',
                    'Imputation/Status Flag For Percent of first-time degree/certificate-seeking students submitting ACT scores (ADM2019)',
                    'Imputation/Status Flag For SAT Evidence-Based Reading and Writing 25th percentile score (ADM2019)',
                    'Imputation/Status Flag For SAT Evidence-Based Reading and Writing 75th percentile score (ADM2019)',
                    'Imputation/Status Flag For SAT Math 25th percentile score (ADM2019)',
                    'Imputation/Status Flag For SAT Math 75th percentile score (ADM2019)',
                    'Imputation/Status Flag For ACT Composite 25th percentile score (ADM2019)',
                    'Imputation/Status Flag For ACT Composite 75th percentile score (ADM2019)',
                    'Imputation/Status Flag For ACT English 25th percentile score (ADM2019)',
                    'Imputation/Status Flag For ACT English 75th percentile score (ADM2019)',
                    'Imputation/Status Flag For ACT Math 25th percentile score (ADM2019)',
                    'Imputation/Status Flag For ACT Math 75th percentile score (ADM2019)', 'Unnamed: 33'
                   ]

In [14]:
data.drop(columns = columns_to_drop2, inplace = True)

In [15]:
data.shape

(1947, 23)

#### Loading in dataframe with college football conference information. Like the admissions statistics, used the 'ipeds_id' to search for this on https://nces.ed.gov/ipeds/use-the-data

In [16]:
sports = pd.read_csv('./datasets/sports.csv')
sports.head()

,UnitID,Institution Name,NCAA/NAIA member for football (IC2019),NCAA/NAIA conference number football (IC2019)
0,222178,Abilene Christian University,Yes,Southland Conference
1,138558,Abraham Baldwin Agricultural College,No,Not applicable
2,108232,Academy of Art University,No,Not applicable
3,126182,Adams State University,Yes,Rocky Mountain Athletic Conference
4,188429,Adelphi University,No,Not applicable


In [17]:
sports.drop(columns = ['NCAA/NAIA member for football (IC2019)', 'Institution Name'], axis = 1, inplace = True)

#### Merging in college football information to base dataframe

In [18]:
sports['UnitID'].astype(int)
data = data.merge(sports, left_on = data['ipeds_id'].astype(int), right_on = sports['UnitID'].astype(int), how = 'left')

In [19]:
data.shape

(1947, 26)

#### Loading in regions dataframe

In [20]:
regions = pd.read_csv('datasets/regions.csv')
regions.head()

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [21]:
regions.shape

(51, 4)

#### Dropping columns unnecessary for analysis

In [22]:
regions.drop(columns = ['State Code', 'Division'], inplace = True)

#### Creating dictionary of states with there corresponding regions and mapping dictionary with 'state' to create 'region' column

In [23]:
#replace District of Columbia with Washington D.C to match up with combined dataframe
regions['State'].replace('District of Columbia', 'Washington, D.C.', inplace=True)

#create a dictionary with region values and corresponding states to use to map region to dataframe.
region_dict = pd.Series(data = regions.Region.values, index= regions.State).to_dict()

data['region'] = data['state'].map(region_dict)

#### Loading in state covid policies/mandates dataframe

In [24]:
sp = pd.read_csv('datasets/state_policies.csv')

#### Creating function to streamline this process for other data dumps from KFF (all same format)

In [25]:
def clean_social_distance(df):
    df['Unnamed: 0'].replace('District of Columbia', 'Washington, D.C.', inplace = True)
    df.rename(columns = {'Unnamed: 0': 'state'}, inplace = True)
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.drop([0, 1], axis = 0, inplace = True)
    df.reset_index(drop = True)
    df.sort_values('state')
    
    return df

clean_social_distance(sp)
sp.head()

,state,status_of_reopening,stay_at_home_order,mandatory_quarantine_for_travelers,non-essential_business_closures,large_gatherings_ban,restaurant_limits,bar_closures,face_covering_requirement,primary_election_postponement,emergency_declaration
2,Alabama,Paused,Lifted,-,All Non-Essential Businesses Permitted to Reop...,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,-,Yes
3,Alaska,Proceeding with Reopening,Lifted,All Travelers,All Non-Essential Businesses Permitted to Reopen,Lifted,Reopened to Dine-in Service,Reopened,Required for Certain Employees,-,Yes
4,Arizona,New Restrictions Imposed,Lifted,Lifted,New Business Closures or Limits,New Limit on Large Gatherings in Place,New Service Limits,Newly Closed,Required for Certain Employees;\nAllows Local ...,-,Yes
5,Arkansas,Paused,-,Lifted,-,Lifted,Reopened to Dine-in Service with Capacity Limits,Reopened,Required for General Public,-,Yes
6,California,New Restrictions Imposed,Statewide,-,New Business Closures or Limits,All Gatherings Prohibited,Newly Closed to Dine-in Service,Newly Closed,Required for General Public,-,Yes


#### Dropping columns unnecessary for analysis

In [26]:
columns_to_drop3 = [
                    'mandatory_quarantine_for_travelers',
                    'non-essential_business_closures',
                    'primary_election_postponement',
                    'emergency_declaration'
                   ]

# 'status_of_reopening'

In [27]:
sp.drop(columns = columns_to_drop3, inplace = True)

#### Merging in state policies to base dataframe

In [28]:
data = data.merge(sp, left_on = ['state'], right_on = ['state'], how = 'right')

In [29]:
data.shape

(1929, 33)

#### Loading in-person classes plan dataframe

In [30]:
cp = pd.read_csv('./datasets/class_plan.csv')

In [31]:
cp.head()

,college,class_plan,total_enrollment
0,Abilene Christian University,Primarily in person,5204
1,Abraham Baldwin Agricultural College,Primarily online,4291
2,Adams State University,Hybrid,3110
3,Adelphi University,Primarily online,8146
4,Adrian College,Primarily in person,1856


In [32]:
data['college2'] = data['college'].str.lower()
cp['college2'] = cp['college'].str.lower()
cp['college2'] = cp['college2'].astype(str)

#### Creating function to XXXX

In [33]:
def remove_space(column):
    
    college_list = []
    
    for college in column:
        
        cl = college.split()
        
        if 'at' in cl:
            cl.remove('at')
        if 'in' in cl:
            cl.remove('in')    
        if 'the' in cl:
            cl.remove('the')
        
        new_college = "".join(cl)
        new_college = "".join(new_college.split(','))
        new_college = "".join(new_college.split('.'))
        new_college = "".join(new_college.split('-'))
        new_college = "".join(new_college.split('/'))
        new_college = "".join(new_college.split("'"))
    
        college_list.append(new_college)
    
    return college_list

In [34]:
data['college2'] = remove_space(data['college2'])
cp['college2'] = remove_space(cp['college2'])

In [35]:
data = data.merge(cp, on = 'college2', how = 'inner')

In [36]:
data.columns

Index(['key_0', 'state', 'ipeds_id', 'college_x', 'cases', 'ids',
       'Institution size category (HD2019)',
       'Distance education courses offered (IC2019)',
       'Admissions total (ADM2019)', 'Enrolled total (ADM2019)',
       'Admissions men (ADM2019)', 'Admissions women (ADM2019)',
       'Number of first-time degree/certificate-seeking students submitting SAT scores (ADM2019)',
       'Number of first-time degree/certificate-seeking students submitting ACT scores (ADM2019)',
       'SAT Evidence-Based Reading and Writing 25th percentile score (ADM2019)',
       'SAT Evidence-Based Reading and Writing 75th percentile score (ADM2019)',
       'SAT Math 25th percentile score (ADM2019)',
       'SAT Math 75th percentile score (ADM2019)',
       'ACT Composite 25th percentile score (ADM2019)',
       'ACT Composite 75th percentile score (ADM2019)',
       'ACT English 25th percentile score (ADM2019)',
       'ACT English 75th percentile score (ADM2019)',
       'ACT Math 25th p

In [37]:
data.drop(columns = ['key_0', 'college2', 'college_y', 'UnitID'], inplace = True)

In [38]:
data.shape

(1382, 33)

## Data Cleaning

In [39]:
data.isnull().sum()

state                                                                                         0
ipeds_id                                                                                      0
college_x                                                                                     0
cases                                                                                        37
ids                                                                                           0
Institution size category (HD2019)                                                            0
Distance education courses offered (IC2019)                                                   0
Admissions total (ADM2019)                                                                  195
Enrolled total (ADM2019)                                                                    195
Admissions men (ADM2019)                                                                    213
Admissions women (ADM2019)              

#### Dropping all rows with null values

In [40]:
data.dropna(inplace = True)

In [41]:
data.shape

(877, 33)

#### Renaming columns

In [42]:
column_names_dict = {
                    'college_x': 'college',
                    'Institution size category (HD2019)' : 'college_size_category',
                    'Distance education courses offered (IC2019)' : 'distance_educ_offered',
                    'Admissions total (ADM2019)' : 'admissions_total', 
                    'Enrolled total (ADM2019)' : 'freshmen_enrollment',
                    'Admissions men (ADM2019)' : 'admissions_men', 
                    'Admissions women (ADM2019)' : 'admissions_women',
                    'Number of first-time degree/certificate-seeking students submitting SAT scores (ADM2019)' : 'number_freshmen_submitting_sat',
                    'Number of first-time degree/certificate-seeking students submitting ACT scores (ADM2019)' : 'number_freshmen_submitting_act',
                    'SAT Evidence-Based Reading and Writing 25th percentile score (ADM2019)' : 'sat_ebrw_25',
                    'SAT Evidence-Based Reading and Writing 75th percentile score (ADM2019)' : 'sat_ebrw_75',
                    'SAT Math 25th percentile score (ADM2019)' : 'sat_math_25',
                    'SAT Math 75th percentile score (ADM2019)' : 'sat_math_75',
                    'ACT Composite 25th percentile score (ADM2019)' : 'act_comp_25',
                    'ACT Composite 75th percentile score (ADM2019)' : 'act_comp_75',
                    'ACT English 25th percentile score (ADM2019)' : 'act_english_25',
                    'ACT English 75th percentile score (ADM2019)' : 'act_english_75',
                    'ACT Math 25th percentile score (ADM2019)' : 'act_math_25',
                    'ACT Math 75th percentile score (ADM2019)' : 'act_math_75',
                    'NCAA/NAIA conference number football (IC2019)': 'football_conference',
                    'status_of_reopening' : 'status_reopening',
                    'large_gatherings_ban' : 'large_gatherings',
                    'face_covering_requirement' : 'face_coverings'
                    }

In [43]:
data.rename(columns = column_names_dict, inplace = True)

In [44]:
data.head()

,state,ipeds_id,college,cases,ids,college_size_category,distance_educ_offered,admissions_total,freshmen_enrollment,admissions_men,...,football_conference,region,status_reopening,stay_at_home_order,large_gatherings,restaurant_limits,bar_closures,face_coverings,class_plan,total_enrollment
0,Alabama,100654.0,Alabama A&M University,41.0,100654,3.0,1.0,8789.0,1710.0,2828.0,...,Southwestern Athletic Conference,South,Paused,Lifted,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,Hybrid,6106
1,Alabama,100724.0,Alabama State University,2.0,100724,2.0,1.0,6467.0,1023.0,1963.0,...,Southwestern Athletic Conference,South,Paused,Lifted,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,Hybrid,4413
2,Alabama,100858.0,Auburn University,2134.0,100858,5.0,1.0,16300.0,4808.0,6997.0,...,Southeastern Conference,South,Paused,Lifted,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,Hybrid,30440
3,Alabama,100830.0,Auburn University at Montgomery,140.0,100830,3.0,1.0,3716.0,626.0,1306.0,...,Not applicable,South,Paused,Lifted,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,Hybrid,5211
4,Alabama,100937.0,Birmingham-Southern College,214.0,100937,2.0,1.0,1821.0,332.0,842.0,...,Southern Athletic Association,South,Paused,Lifted,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,Primarily in person,1268


#### XXX

In [45]:
data['status_reopening'].value_counts()

New Restrictions Imposed     426
Proceeding with Reopening    236
Paused                       125
Reopened                      90
Name: status_reopening, dtype: int64

In [46]:
data['status_reopening'].replace('Proceeding with Reopening', 'Reopened', inplace = True)

#### XXX

In [47]:
data['large_gatherings'].value_counts()

New Limit on Large Gatherings in Place    253
Expanded to New Limit Above 25            239
Lifted                                    169
Expanded to New Limit of 25 or Fewer       80
>10 People Prohibited                      76
All Gatherings Prohibited                  54
-                                           6
Name: large_gatherings, dtype: int64

In [48]:
data['large_gatherings'].replace('>10 People Prohibited', '>25_prohibited', inplace = True)
data['large_gatherings'].replace('Expanded to New Limit of 25 or Fewer', '>25_prohibited', inplace = True)

#### XXX

In [49]:
data['restaurant_limits'].value_counts()

Reopened to Dine-in Service with Capacity Limits    404
New Service Limits                                  195
Reopened to Dine-in Service                         175
Newly Closed to Dine-in Service                      54
Reopened to Dine-in Service                          40
-                                                     9
Name: restaurant_limits, dtype: int64

In [50]:
data['restaurant_limits'].replace('Reopened to Dine-in Service ', 'Reopened to Dine-in Service', inplace = True)

#### XXX

In [51]:
data['bar_closures'].value_counts()

Reopened              362
Newly Closed          258
Closed                146
New Service Limits    111
Name: bar_closures, dtype: int64

In [52]:
data['bar_closures'].replace('Newly Closed', 'Closed', inplace = True)

#### XXX

In [53]:
data['face_coverings'].value_counts()

Required for General Public                                                              687
Required for Certain Employees                                                            80
Allows Local Officals to Require for General Public                                       58
-                                                                                         44
-                                                                                          6
Required for Certain Employees;\nAllows Local Officials to Require for General Public      2
Name: face_coverings, dtype: int64

In [54]:
data['face_coverings'].replace('- ', '-', inplace = True)
data['face_coverings'].replace('Required for Certain Employees;\nAllows Local Officials to Require for General Public', 'Required for Certain Employees', inplace = True)

#### XXX

In [55]:
data['class_plan'].value_counts()

Primarily in person    272
Primarily online       261
Hybrid                 238
Fully online            65
Fully in person         29
Other                    7
Undetermined             5
Name: class_plan, dtype: int64

In [56]:
data['class_plan'].replace('Undetermined', 'Other', inplace = True)

#### Creating target variable --> Colleges with greater than 5% infection rate

In [57]:
data['percent_infected'] = (data['cases'] / data['total_enrollment'])

In [58]:
data['greater_than_5'] = ((data['cases'] / data['total_enrollment']) > 0.05).astype(int)

In [59]:
data.head()

,state,ipeds_id,college,cases,ids,college_size_category,distance_educ_offered,admissions_total,freshmen_enrollment,admissions_men,...,status_reopening,stay_at_home_order,large_gatherings,restaurant_limits,bar_closures,face_coverings,class_plan,total_enrollment,percent_infected,greater_than_5
1363,Wisconsin,239390.0,Mount Mary University,58.0,239390,2.0,1.0,419.0,126.0,0.0,...,Reopened,Lifted,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,Primarily online,1339,0.043316,0
1369,Wisconsin,240444.0,University of Wisconsin-Madison,5182.0,240444,5.0,1.0,23886.0,7550.0,11118.0,...,Reopened,Lifted,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,Primarily online,43463,0.119228,1
1379,Wisconsin,240107.0,Viterbo University,311.0,240107,2.0,1.0,974.0,283.0,237.0,...,Reopened,Lifted,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,Primarily in person,2598,0.119707,1
1380,Wisconsin,240338.0,Wisconsin Lutheran College,120.0,240338,2.0,1.0,670.0,213.0,316.0,...,Reopened,Lifted,Lifted,Reopened to Dine-in Service,Reopened,Required for General Public,Hybrid,1248,0.096154,1
1381,Wyoming,240727.0,University of Wyoming,1795.0,240727,4.0,1.0,5132.0,1760.0,2569.0,...,Reopened,-,Expanded to New Limit Above 25,Reopened to Dine-in Service,Reopened,Required for Certain Employees,Primarily in person,12450,0.144177,1


#### Final drop of unecessary columns and saving combined dataframe

In [63]:
data.drop(columns = ['ipeds_id', 'ids'], inplace = True)

In [64]:
data.to_csv('./datasets/combined_df2.csv', index = False)